# Problem 2

In [2]:
!pip install wrds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [5]:
import numpy as np
import pandas as pd
import wrds
import os
import matplotlib.pyplot as plt
import random
import warnings
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import glob

In [6]:
warnings.filterwarnings("ignore", category=UserWarning)

SEED = 123
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

## Part (a) (b)

## Data

In [7]:
# fetching data from WRDS
# def get_prices(ticker_list, db, period=('2010-01-01', '2020-12-31')):
#     if not ticker_list:
#       return pd.DataFrame(columns=["ticker", "secid", "effect_date"])
#
#     formatted_tickers = "', '".join(ticker_list)
#     query = f"""
#         SELECT permno, permco,htick, htsymbol
#         FROM crsp_a_stock.dsfhdr
#         WHERE htsymbol IN ('{formatted_tickers}')
#     """
#     df_permno =  db.raw_sql(query)
#     permno_lst = df_permno['permno'].astype(str).to_list()
#     formatted_permno = ",".join(permno_lst)
#     query = f"""
#         SELECT permno, date, prc, cfacpr
#         FROM crsp.dsf
#         WHERE permno IN ({formatted_permno})
#           AND date BETWEEN '{period[0]}' AND '{period[1]}'
#         ORDER BY permno, date;
#         """
#     df = db.raw_sql(query)
#     return df, df_permno
#
# # Read from csv
# # master_df = pd.read_csv('/content/drive/MyDrive/colab_notebooks/230P/problem_set_2/deliverables_q1/deliverables_q2/apple_stock.csv')
#
# db = wrds.Connection(wrds_username='jacoblan')
# df, info = get_prices(["AAPL"],db)
# df.to_parquet('aapl_prices_2010_2020.parquet', index=False)

In [8]:
# read data from parquet
df = pd.read_parquet('aapl_prices_2010_2020.parquet')

df['date'] = pd.to_datetime(df['date'])
df['adj_close'] = df['prc'] / df['cfacpr']
df = df.sort_values('date').reset_index(drop=True)

df.head()

,permno,date,prc,cfacpr,adj_close
0,14593,2010-01-04,214.00999,28.0,7.643214
1,14593,2010-01-05,214.38,28.0,7.656429
2,14593,2010-01-06,210.97,28.0,7.534643
3,14593,2010-01-07,210.58,28.0,7.520714
4,14593,2010-01-08,211.98,28.0,7.570714


## Bollinger Bands for Window Sizes 5 & 20

In [9]:
def compute_bb_labels(df, window, std=1):
    df = df.copy()
    df[f'SMA_{window}'] = df['adj_close'].rolling(window).mean()
    df[f'STD_{window}'] = df['adj_close'].rolling(window).std()
    df[f'Upper_BB_{window}'] = df[f'SMA_{window}'] + std * df[f'STD_{window}']
    df[f'Lower_BB_{window}'] = df[f'SMA_{window}'] - std * df[f'STD_{window}']

    def bb_label(row):
        if row['adj_close'] < row[f'Lower_BB_{window}']:
            return 'buy'
        elif row['adj_close'] > row[f'Upper_BB_{window}']:
            return 'sell'
        else:
            return None

    df[f'BB_label_{window}'] = df.apply(bb_label, axis=1)
    return df.dropna(subset=[f'BB_label_{window}'])

## Short-Term Future Return Labels for 5 & 20 Days

In [10]:
def compute_future_labels(df, horizon):
    """
    Labels each row based on the price change from time t to t+horizon,
    following: +2% => 'buy', -2% => 'sell', else None.
    """
    df = df.copy()
    label_col = f'Future_label_{horizon}'
    df[label_col] = None

    for i in range(len(df) - horizon):
        start_price = df.iloc[i]['adj_close']
        end_price = df.iloc[i + horizon]['adj_close']
        pct_change = end_price / start_price - 1
        if pd.isnull(start_price) or pd.isnull(end_price):
            continue
        if pct_change >= 0.02:
            df.at[df.index[i], label_col] = 'buy'
        elif pct_change <= -0.02:
            df.at[df.index[i], label_col] = 'sell'

    return df.dropna(subset=[label_col])


## Save Price Charts for Each Configuration

In [11]:
def save_price_charts_bb(df, window_size, label_col, base_dir,
                         price_color='black', upper_color='blue', lower_color='red', line_width=1):

    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(f"{base_dir}/buy", exist_ok=True)
    os.makedirs(f"{base_dir}/sell", exist_ok=True)

    upper_col = f'Upper_BB_{window_size}'
    lower_col = f'Lower_BB_{window_size}'

    counter = 0
    for i in range(window_size - 1, len(df)):
        window_df = df.iloc[i - window_size + 1: i + 1]
        label = df.iloc[i][label_col]
        if label not in ['buy', 'sell']:
            continue

        start_date = window_df.iloc[0]['date'].strftime('%Y%m%d')
        end_date = window_df.iloc[-1]['date'].strftime('%Y%m%d')
        filename = f"{base_dir}/{label}/img_{counter}_{start_date}_{end_date}.png"

        plt.figure(figsize=(2, 2))
        plt.plot(window_df['adj_close'].reset_index(drop=True), color=price_color, lw=line_width, label='Adj Close')
        plt.plot(window_df[upper_col].reset_index(drop=True), color=upper_color, lw=line_width, linestyle='--', label='Upper BB')
        plt.plot(window_df[lower_col].reset_index(drop=True), color=lower_color, lw=line_width, linestyle='--', label='Lower BB')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(filename, dpi=100)
        plt.close()
        counter += 1

def save_price_charts_future(df, window_size, label_col, base_dir,
                             price_color='black', up_line_color='green', down_line_color='orange', line_width=1):
    """
    For each labeled row, plots the price window from t to t+horizon,
    including 2% threshold lines, and saves under buy/sell folders.
    """

    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(f"{base_dir}/buy", exist_ok=True)
    os.makedirs(f"{base_dir}/sell", exist_ok=True)

    counter = 0
    for i in range(len(df) - window_size):
        label = df.iloc[i][label_col]
        if label not in ['buy', 'sell']:
            continue

        window_df = df.iloc[i : i + window_size + 1].copy()
        start_date = window_df.iloc[0]['date'].strftime('%Y%m%d')
        end_date = window_df.iloc[-1]['date'].strftime('%Y%m%d')
        filename = f"{base_dir}/{label}/img_{counter}_{start_date}_{end_date}.png"

        base = window_df.iloc[0]['adj_close']
        up_2 = base * 1.02
        down_2 = base * 0.98

        plt.figure(figsize=(2, 2))
        plt.plot(window_df['adj_close'].reset_index(drop=True), color=price_color, lw=line_width)
        plt.axhline(up_2, color=up_line_color, lw=line_width, linestyle='--')
        plt.axhline(down_2, color=down_line_color, lw=line_width, linestyle='--')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(filename, dpi=100)
        plt.close()
        counter += 1


## BB and Future Gain Graphs — Window 5 and 20

In [12]:
# For Bollinger Bands labeling (keep your original compute_bb_labels code unchanged)
df_bb_5 = compute_bb_labels(df, window=5)
save_price_charts_bb(df_bb_5, window_size=5, label_col='BB_label_5', base_dir='graphs_bb_win5')

df_bb_20 = compute_bb_labels(df, window=20)
save_price_charts_bb(df_bb_20, window_size=20, label_col='BB_label_20', base_dir='graphs_bb_win20')

# For short-term return labeling
df_fut_5 = compute_future_labels(df, horizon=5)
save_price_charts_future(df_fut_5, window_size=5, label_col='Future_label_5', base_dir='graphs_fut_win5')

df_fut_20 = compute_future_labels(df, horizon=20)
save_price_charts_future(df_fut_20, window_size=20, label_col='Future_label_20', base_dir='graphs_fut_win20')

In [13]:
# one error example for future gains 20 days
df[(df['date'] > '2010-05-02') & (df['date'] < '2010-06-03')]

,permno,date,prc,cfacpr,adj_close
82,14593,2010-05-03,266.35001,28.0,9.5125
83,14593,2010-05-04,258.67999,28.0,9.238571
84,14593,2010-05-05,255.985,28.0,9.142321
85,14593,2010-05-06,246.25,28.0,8.794643
86,14593,2010-05-07,235.86,28.0,8.423571
87,14593,2010-05-10,253.99001,28.0,9.071072
88,14593,2010-05-11,256.51999,28.0,9.161428
89,14593,2010-05-12,262.09,28.0,9.360357
90,14593,2010-05-13,258.35999,28.0,9.227142
91,14593,2010-05-14,253.82001,28.0,9.065


## CNN Model

In [20]:
SEED = 123
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

img_size = (64, 64)

def get_image_and_labels(base_dir):
    X, y = [], []
    for label in ['buy', 'sell']:
        img_paths = glob.glob(f"{base_dir}/{label}/*.png")
        X.extend(img_paths)
        y.extend([0 if label == 'buy' else 1] * len(img_paths))
    return np.array(X), np.array(y)

def split_dataset(X, y, seed=SEED):
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, random_state=seed, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=2/3, random_state=seed, stratify=y_temp)
    return X_train, y_train, X_val, y_val, X_test, y_test

def preprocess_image(filepath):
    img = tf.keras.utils.load_img(filepath, color_mode='rgb', target_size=img_size)
    img = tf.keras.utils.img_to_array(img)
    img = img / 255.0
    return img

def build_dataset(X, y):
    imgs = np.array([preprocess_image(fp) for fp in X])
    return imgs, np.array(y)

# Global results list
model_results = []

def train_and_evaluate(X, y, model_name):
    X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(X, y, seed=SEED)
    X_train_img, y_train = build_dataset(X_train, y_train)
    X_val_img, y_val     = build_dataset(X_val, y_val)
    X_test_img, y_test   = build_dataset(X_test, y_test)

    model = models.Sequential([
        Input(shape=(img_size[0], img_size[1], 3)),
        layers.Conv2D(16, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=5)
    ]
    model.fit(
        X_train_img, y_train,
        validation_data=(X_val_img, y_val),
        epochs=25,
        batch_size=32,
        shuffle=True,
        callbacks=callbacks,
        verbose=0
    )

    # Evaluate
    y_train_pred = (model.predict(X_train_img, verbose=0) > 0.5).astype(int).flatten()
    y_test_pred  = (model.predict(X_test_img, verbose=0)  > 0.5).astype(int).flatten()
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc  = accuracy_score(y_test,  y_test_pred)

    model_results.append({
        "Model": model_name,
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc,
    })

    # (Keep your existing detailed output for full results if desired...)
    label_map = {0: "buy", 1: "sell"}
    print(f"\n{model_name}:")
    print(f"  Train accuracy: {train_acc:.3f}")
    print(f"  Test  accuracy: {test_acc:.3f}")
    print(f"\nTest results for {model_name}:")
    print(f"{'Test Image':40} | {'Predicted':8} | {'Actual':8}")
    print('-'*65)
    for fname, pred, actual in zip(X_test, y_test_pred, y_test):
        print(f"{os.path.basename(fname):40} | {label_map[pred]:8} | {label_map[actual]:8}")

    model.save(f"{model_name}.keras")
    np.savez(f"{model_name}_test_results.npz", X_test=X_test, y_test=y_test, y_pred=y_test_pred)

    return model

tasks = [
    ("BB_5",    "graphs_bb_win5"),
    ("BB_20",   "graphs_bb_win20"),
    ("Future_5","graphs_fut_win5"),
    ("Future_20","graphs_fut_win20")
]

for model_name, base_dir in tasks:
    if not os.path.exists(base_dir):
        print(f"Directory {base_dir} does not exist, skipping.")
        continue
    X, y = get_image_and_labels(base_dir)
    if len(X) == 0:
        print(f"No images found in {base_dir}")
        continue
    train_and_evaluate(X, y, model_name)


BB_5:
  Train accuracy: 0.997
  Test  accuracy: 0.965

Test results for BB_5:
Test Image                               | Predicted | Actual  
-----------------------------------------------------------------
img_1174_20191230_20200109.png           | sell     | sell    
img_134_20110208_20110216.png            | sell     | sell    
img_1004_20180727_20180813.png           | sell     | sell    
img_967_20180412_20180420.png            | buy      | buy     
img_714_20160216_20160302.png            | sell     | sell    
img_136_20110214_20110222.png            | buy      | buy     
img_674_20150928_20151008.png            | sell     | buy     
img_187_20110720_20110802.png            | buy      | buy     
img_511_20140508_20140523.png            | sell     | sell    
img_846_20170301_20170320.png            | sell     | sell    
img_602_20150123_20150204.png            | sell     | sell    
img_818_20161201_20161209.png            | sell     | sell    
img_388_20130401_20130417.png      

In [21]:
# Summary of accuracy result
results_df = pd.DataFrame(model_results)
print("\nSummary of Model Accuracies:")
print(results_df.to_string(index=False))


Summary of Model Accuracies:
    Model  Train Accuracy  Test Accuracy
     BB_5        0.996644       0.964844
    BB_20        1.000000       0.951299
 Future_5        0.987144       0.974359
Future_20        0.987467       0.979263


#Summary
We start by turning every 5- or 20-day sliding window of AAPL adjusted-close prices into a 128 × 128 RGB chart that shows the price curve (plus its Bollinger-Band or ±2 % reference lines) and save it as a PNG. The colour channels are preserved so the CNN can learn that “blue dashed” means an upper band and “red dashed” a lower band instead of inferring that information from intensity alone. File paths are collected, stratified into a 70-10-20 random train/val/test split, and each image is loaded with tf.keras.utils.load_img(..., color_mode='rgb'), rescaled to [0,1], and stacked into tensors. The classification model is a small four-layer CNN: an input layer expecting (128,128,3), two conv–max-pool blocks (16 and 32 filters, 3×3 kernels, ReLU), a Flatten, one 32-unit dense layer, and a 1-unit sigmoid output. We compile it with Adam and binary cross-entropy, and train for up to 50 epochs with EarlyStopping (patience = 10, best-weight restore) so training halts automatically once validation loss stops improving. Finally, we evaluate on the held-out test set.

## Part (c)
Discuss why designing models using a random split in stock forecasting is problematic.

Designing models using a random split in stock forecasting is problematic primarily because it introduces ***look-ahead bias*** and leads to information leakage between the training and test sets. In financial time series like stock returns, the data is highly dependent on the order of time, future prices are influenced by past prices, not vice versa. A random split can mix data from different time periods in training, validation, and test sets, making it possible for the model to “see” patterns from the future during training.
As a result, the model’s performance may appear overly optimistic, since it’s being evaluated on data that, in reality, would not be known at prediction time. This misrepresents the model’s true predictive power in a real-world scenario, where only past and present data are available when forecasting future prices. Therefore, it is more appropriate to use chronological splits (e.g., train on earlier dates, validate and test on later dates) to ensure the model’s performance reflects its ability to generalize to unseen, future data—just like in actual trading.